#### import packages/ data preps

In [99]:
import csv
import jellyfish
import pandas as pd
import util

In [100]:
# Hard-coded filename
zagat_filename = "data/zagat.csv"
fodors_filename = "data/fodors.csv"
known_links_filename = "data/known_links.csv"

### YOUR CODE HERE
za_df = pd.read_csv(zagat_filename)
fo_df = pd.read_csv(fodors_filename)
known_links_df = pd.read_csv(known_links_filename,header=None)
unmatch_pairs_df = pd.read_csv("data/unmatch_pairs.csv")

In [101]:
known_links_df.head()

,0,1
0,269,386
1,106,145
2,21,19
3,20,18
4,142,243


In [103]:
za_df

,index,restaurant name,city,street address
0,0,Apple Pan The,West LA,10801 W. Pico Blvd.
1,1,Arnie Morton's of Chicago,Los Angeles,435 S. La Cienega Blvd.
2,2,Art's Deli,Studio City,12224 Ventura Blvd.
3,3,Asahi Ramen,West LA,2027 Sawtelle Blvd.
4,4,Baja Fresh,Westlake Village,3345 Kimber Dr.
...,...,...,...,...
326,326,Ti Couz,San Francisco,3108 16th St.
327,327,Trio Cafe,San Francisco,1870 Fillmore St.
328,328,Tu Lan,San Francisco,8 Sixth St.
329,329,Vicolo Pizzeria,San Francisco,201 Ivy St.


In [104]:
fo_df

,index,restaurant name,city,street address
0,0,Adriano's Ristorante,Los Angeles,2930 Beverly Glen Circle
1,1,Arnie Morton's of Chicago,Los Angeles,435 S. La Cienega Blvd.
2,2,Art's Delicatessen,Studio City,12224 Ventura Blvd.
3,3,Barney Greengrass,Beverly Hills,9570 Wilshire Blvd.
4,4,Beaurivage,Malibu,26025 Pacific Coast Hwy.
...,...,...,...,...
528,528,Yank Sing,San Francisco,427 Battery St.
529,529,Yaya Cuisine,San Francisco,1220 9th Ave.
530,530,Yoyo Tsumami Bistro,San Francisco,1611 Post St.
531,531,Zarzuela,San Francisco,2000 Hyde St.


#### 3x3x3 - ? dictionary

In [105]:
def construct_similarity_tuple_dictionary():
    '''
    This function initializes a dictionary to store the frequency of each similarity tuple.
    '''
    result = {}
    for a in ['low', 'medium', 'high']:
        for b in ['low', 'medium', 'high']:
            for c in ['low', 'medium', 'high']:
                result[(a, b, c)] = 0
    return result

#### compute_similarity_ctg & compute_similarity_tpl(za_series, fo_series)

In [112]:
def compute_similarity_ctg(str1, str2):
    '''
    This function computes the Jaro-Winkler similarity between two strings 
    and categorizes the result into "low", "medium", or "high" 
    '''
    jw_score = jellyfish.jaro_winkler_similarity(str(str1), str(str2))
    jw_ctg = util.get_jw_category(jw_score)
    return jw_ctg

In [113]:
type(fo_df.iloc[0])

pandas.core.series.Series

In [228]:
za_series = za_df.iloc[0]
za_series

index                                0
restaurant name         Apple Pan  The
city                           West LA
street address     10801 W. Pico Blvd.
Name: 0, dtype: object

In [217]:
za_series = za_df.iloc[0][1:]
fo_series = fo_df.iloc[0][1:]
za_series

index                                0
restaurant name         Apple Pan  The
city                           West LA
street address     10801 W. Pico Blvd.
Name: 0, dtype: object

In [231]:
def compute_similarity_tpl(za_series, fo_series):
    '''
    This is a helper function that constructs a similarity tuple for a pair of rows from the Zagat and Fodor's datasets. 
    It computes the similarity category for the name, city, and address fields.
    '''    
    similarity_tpl = ()
    for i in range(1,4):
        ctg = compute_similarity_ctg(za_series.iloc [i], fo_series.iloc[i])
        similarity_tpl += (ctg,)
    return similarity_tpl

In [233]:
compute_similarity_tpl(za_series, fo_series)

('low', 'low', 'low')

#### estimate_prob(link_df, left_df, right_df)

In [234]:
def estimate_prob(link_df, left_df, right_df):
    similarity_tuple_dict = construct_similarity_tuple_dictionary()
    prob_dict = {}
    for _, row in link_df.iterrows():
        za_idx = row[0]  # First column
        fo_idx = row[1]  # Second column
        
        jw_ctg = compute_similarity_tpl(left_df.iloc[za_idx], 
                                      right_df.iloc[fo_idx])
        similarity_tuple_dict[jw_ctg] += 1
        
    for key, val in similarity_tuple_dict.items():
        prob_dict[key] = val/len(link_df)
        
    return prob_dict


In [236]:
estimate_prob(known_links_df, za_df, fo_df)

{('low', 'low', 'low'): 0.0,
 ('low', 'low', 'medium'): 0.0,
 ('low', 'low', 'high'): 0.0,
 ('low', 'medium', 'low'): 0.0,
 ('low', 'medium', 'medium'): 0.02,
 ('low', 'medium', 'high'): 0.0,
 ('low', 'high', 'low'): 0.0,
 ('low', 'high', 'medium'): 0.0,
 ('low', 'high', 'high'): 0.06,
 ('medium', 'low', 'low'): 0.02,
 ('medium', 'low', 'medium'): 0.0,
 ('medium', 'low', 'high'): 0.02,
 ('medium', 'medium', 'low'): 0.0,
 ('medium', 'medium', 'medium'): 0.0,
 ('medium', 'medium', 'high'): 0.06,
 ('medium', 'high', 'low'): 0.0,
 ('medium', 'high', 'medium'): 0.02,
 ('medium', 'high', 'high'): 0.08,
 ('high', 'low', 'low'): 0.0,
 ('high', 'low', 'medium'): 0.0,
 ('high', 'low', 'high'): 0.0,
 ('high', 'medium', 'low'): 0.06,
 ('high', 'medium', 'medium'): 0.06,
 ('high', 'medium', 'high'): 0.2,
 ('high', 'high', 'low'): 0.02,
 ('high', 'high', 'medium'): 0.1,
 ('high', 'high', 'high'): 0.28}

In [237]:
mat_prob_dict = estimate_prob(known_links_df, za_df, fo_df)
unm_prob_dict = estimate_prob(unmatch_pairs_df,za_df, fo_df)

/var/folders/7h/mx7nc7vj63551qp5lczzf1lr0000gn/T/ipykernel_10502/2026111112.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  za_idx = row[0]  # First column
/var/folders/7h/mx7nc7vj63551qp5lczzf1lr0000gn/T/ipykernel_10502/2026111112.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fo_idx = row[1]  # Second column


#### initial_screening_possible_match(mat_prob_dict, unm_prob_dict)

In [238]:
def initial_screening_possible_match(mat_prob_dict, unm_prob_dict):
    partition_dict = construct_similarity_tuple_dictionary()
    possible_match = []
    all = partition_dict.keys()
    for tpl in all: 
        m_w = mat_prob_dict[tpl]
        u_w = unm_prob_dict[tpl]
        if  m_w == 0 and u_w == 0:
            possible_match.append(tpl)
    return possible_match

In [239]:
possible = initial_screening_possible_match(mat_prob_dict, unm_prob_dict)
possible

[('low', 'low', 'medium'),
 ('low', 'low', 'high'),
 ('low', 'medium', 'high'),
 ('medium', 'low', 'medium'),
 ('medium', 'medium', 'medium'),
 ('medium', 'high', 'low'),
 ('high', 'low', 'low'),
 ('high', 'low', 'medium'),
 ('high', 'low', 'high')]

#### construct_probability_tuples(mat_prob_dict, unm_prob_dict)

In [240]:
def construct_probability_tuples(mat_prob_dict, unm_prob_dict):
    partition_dict = construct_similarity_tuple_dictionary()
    all = partition_dict.keys()
    possible_match = initial_screening_possible_match(mat_prob_dict, unm_prob_dict)
    unvisited = all - possible_match

    for_sorting = []
    for key in unvisited:
        for_sorting.append((key, mat_prob_dict[key], unm_prob_dict[key]))
    
    sorted_remaining = util.sort_prob_tuples(for_sorting)
    return sorted_remaining

In [241]:
sorted_remaining = construct_probability_tuples(mat_prob_dict, unm_prob_dict)
sorted_remaining

[(('high', 'high', 'high'), 0.28, 0.0),
 (('high', 'medium', 'high'), 0.2, 0.0),
 (('high', 'high', 'medium'), 0.1, 0.0),
 (('medium', 'high', 'high'), 0.08, 0.0),
 (('high', 'medium', 'low'), 0.06, 0.0),
 (('high', 'medium', 'medium'), 0.06, 0.0),
 (('low', 'high', 'high'), 0.06, 0.0),
 (('medium', 'medium', 'high'), 0.06, 0.0),
 (('high', 'high', 'low'), 0.02, 0.0),
 (('medium', 'high', 'medium'), 0.02, 0.0),
 (('medium', 'low', 'high'), 0.02, 0.0),
 (('medium', 'low', 'low'), 0.02, 0.001001001001001001),
 (('low', 'medium', 'medium'), 0.02, 0.009009009009009009),
 (('low', 'high', 'low'), 0.0, 0.06006006006006006),
 (('low', 'high', 'medium'), 0.0, 0.003003003003003003),
 (('low', 'low', 'low'), 0.0, 0.7897897897897898),
 (('low', 'medium', 'low'), 0.0, 0.13613613613613615),
 (('medium', 'medium', 'low'), 0.0, 0.001001001001001001)]

#### ini_find_matches(sorted_remaining, mu, lambda_)

In [242]:
def ini_find_matches(sorted_remaining, mu, lambda_):
    cum_u_w = 0
    match = []
    
    unmatch = []
    cum_m_w = 0

    for tpl in sorted_remaining:
        if cum_u_w + tpl[2] > mu:
            break
        cum_u_w += tpl[2]
        match.append(tpl[0])
    
    for tpl in sorted_remaining[::-1]:
        if cum_m_w + tpl[1] > lambda_: 
            break
        cum_m_w += tpl[1]
        unmatch.append(tpl[0])
    return match, unmatch

In [243]:
match, unmatch =  ini_find_matches(sorted_remaining, 0.005, 0.005)

In [244]:
match

[('high', 'high', 'high'),
 ('high', 'medium', 'high'),
 ('high', 'high', 'medium'),
 ('medium', 'high', 'high'),
 ('high', 'medium', 'low'),
 ('high', 'medium', 'medium'),
 ('low', 'high', 'high'),
 ('medium', 'medium', 'high'),
 ('high', 'high', 'low'),
 ('medium', 'high', 'medium'),
 ('medium', 'low', 'high'),
 ('medium', 'low', 'low')]

In [245]:
unmatch

[('medium', 'medium', 'low'),
 ('low', 'medium', 'low'),
 ('low', 'low', 'low'),
 ('low', 'high', 'medium'),
 ('low', 'high', 'low')]

#### check_unlabeled(possible, match, unmatch)

In [246]:
def check_unlabeled(possible, match, unmatch):
    pm = 0
    m = 0
    u = 0
    mapping = {}
    partition_dict = construct_similarity_tuple_dictionary()
    all = partition_dict.keys()
    new_possible = possible.copy()  # Start with initial possible matches
    
    # Handle crossover - bias towards matches
    duplicates = set(match) & set(unmatch)

    if duplicates:
        for dup in duplicates:
                unmatch.remove(dup)
    
    # Find missing/unlabeled tuples and add them to possible matches
    labeled = set(match + unmatch + new_possible)
    for tpl in all:
        if tpl not in labeled:
            new_possible.append(tpl)
    
    # Then apply classifications in priority order
    for pos in new_possible:
        mapping[pos] = 'possible match'
        pm += 1
    for unm in unmatch:
        mapping[unm] = 'unmatch'
        u += 1
    for mat in match:
        mapping[mat] = 'match'
        m += 1
    
    print('m', m)
    print('pm ', pm)
    print('u', u)
    
    return mapping

In [247]:
mapping = check_unlabeled(possible, match, unmatch)
mapping

m 12
pm  10
u 5


{('low', 'low', 'medium'): 'possible match',
 ('low', 'low', 'high'): 'possible match',
 ('low', 'medium', 'high'): 'possible match',
 ('medium', 'low', 'medium'): 'possible match',
 ('medium', 'medium', 'medium'): 'possible match',
 ('medium', 'high', 'low'): 'possible match',
 ('high', 'low', 'low'): 'possible match',
 ('high', 'low', 'medium'): 'possible match',
 ('high', 'low', 'high'): 'possible match',
 ('low', 'medium', 'medium'): 'possible match',
 ('medium', 'medium', 'low'): 'unmatch',
 ('low', 'medium', 'low'): 'unmatch',
 ('low', 'low', 'low'): 'unmatch',
 ('low', 'high', 'medium'): 'unmatch',
 ('low', 'high', 'low'): 'unmatch',
 ('high', 'high', 'high'): 'match',
 ('high', 'medium', 'high'): 'match',
 ('high', 'high', 'medium'): 'match',
 ('medium', 'high', 'high'): 'match',
 ('high', 'medium', 'low'): 'match',
 ('high', 'medium', 'medium'): 'match',
 ('low', 'high', 'high'): 'match',
 ('medium', 'medium', 'high'): 'match',
 ('high', 'high', 'low'): 'match',
 ('medium', 'h

In [248]:
match, unmatch =  ini_find_matches(sorted_remaining, 0.0, 0.01)
mapping = check_unlabeled(possible, match, unmatch)


m 11
pm  11
u 5


In [249]:
match, unmatch =  ini_find_matches(sorted_remaining, 0.0, 0.03)
mapping = check_unlabeled(possible, match, unmatch)


m 11
pm  10
u 6


In [250]:
match, unmatch =  ini_find_matches(sorted_remaining, 0.005, 0.05)
mapping = check_unlabeled(possible, match, unmatch)

m 12
pm  9
u 6


#### find_matches(output_filename, mu, lambda_, block_on_city=False):


In [251]:
def find_matches(output_filename, mu, lambda_, block_on_city=False):
    '''
    Put it all together: read the data and apply the record linkage
    algorithm to classify the potential matches.

    Inputs:
      output_filename (string): the name of the output file,
      mu (float) : the maximum false positive rate,
      lambda_ (float): the maximum false negative rate,
      block_on_city (boolean): indicates whether to block on the city or not.
    '''

    # Hard-coded filename
    zagat_filename = "data/zagat.csv"
    fodors_filename = "data/fodors.csv"
    known_links_filename = "data/known_links.csv"

    ### YOUR CODE HERE
    za_df = pd.read_csv(zagat_filename)
    fo_df = pd.read_csv(fodors_filename)
    known_links_df = pd.read_csv(known_links_filename)
    unmatch_pairs_df = pd.read_csv("data/unmatch_pairs.csv")
    
    # Estimate probabilities
    mat_prob_dict = estimate_prob(known_links_df, za_df, fo_df)
    unm_prob_dict = estimate_prob(unmatch_pairs_df, za_df, fo_df)

    # Construct and sort probability tuples
    sorted_remaining = construct_probability_tuples(mat_prob_dict, unm_prob_dict)

    # Label similarity tuples
    match, unmatch = ini_find_matches(sorted_remaining, mu, lambda_)
    possible = initial_screening_possible_match(mat_prob_dict, unm_prob_dict)
    labels = check_unlabeled(possible, match, unmatch)

    # Classify pairs and write to output file
    with open(output_filename, 'w') as f:
        writer = csv.writer(f)
        for za_index, za_row in za_df.iterrows():
            for fo_index, fo_row in fo_df.iterrows():
                if block_on_city and za_row['city'] != fo_row['city']:
                    continue
                similarity_tpl = compute_similarity_tpl(za_row, fo_row)
                label = labels[similarity_tpl]
                writer.writerow([za_index, fo_index, label])

In [252]:
find_matches('test.csv', 0.005, 0.005, block_on_city=False)

/var/folders/7h/mx7nc7vj63551qp5lczzf1lr0000gn/T/ipykernel_10502/2026111112.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  za_idx = row[0]  # First column
/var/folders/7h/mx7nc7vj63551qp5lczzf1lr0000gn/T/ipykernel_10502/2026111112.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  fo_idx = row[1]  # Second column


m 12
pm  10
u 5


Row: Pandas(Index=0, _1=106, _2=145)
Row: Pandas(Index=1, _1=21, _2=19)
Row: Pandas(Index=2, _1=20, _2=18)
Row: Pandas(Index=3, _1=142, _2=243)
Row: Pandas(Index=4, _1=1, _2=1)
Row: Pandas(Index=5, _1=166, _2=294)
Row: Pandas(Index=6, _1=101, _2=132)
Row: Pandas(Index=7, _1=253, _2=370)
Row: Pandas(Index=8, _1=2, _2=2)
Row: Pandas(Index=9, _1=245, _2=363)
Row: Pandas(Index=10, _1=95, _2=98)
Row: Pandas(Index=11, _1=278, _2=397)
Row: Pandas(Index=12, _1=303, _2=479)
Row: Pandas(Index=13, _1=207, _2=428)
Row: Pandas(Index=14, _1=81, _2=71)
Row: Pandas(Index=15, _1=260, _2=380)
Row: Pandas(Index=16, _1=308, _2=486)
Row: Pandas(Index=17, _1=192, _2=412)
Row: Pandas(Index=18, _1=256, _2=376)
Row: Pandas(Index=19, _1=99, _2=119)
Row: Pandas(Index=20, _1=41, _2=66)
Row: Pandas(Index=21, _1=317, _2=504)
Row: Pandas(Index=22, _1=129, _2=209)
Row: Pandas(Index=23, _1=5, _2=36)
Row: Pandas(Index=24, _1=15, _2=10)
Row: Pandas(Index=25, _1=159, _2=276)
Row: Pandas(Index=26, _1=113, _2=170)
Row: Pan